In [1]:
from DiffFlatQuad.robot import PlanerQuadrotor

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
robot = PlanerQuadrotor(rendering=False)

In [3]:
robot.getSymbolicF()

Matrix([
[x2(t)],
[    0],
[x4(t)],
[   -g],
[x6(t)],
[    0]])

In [4]:
robot.getSymbolicG()

Matrix([
[           0,   0],
[sin(x5(t))/m,   0],
[           0,   0],
[cos(x5(t))/m,   0],
[           0,   0],
[           0, 1/J]])

In [5]:
robot.symbolic_state

Matrix([
[x1(t)],
[x2(t)],
[x3(t)],
[x4(t)],
[x5(t)],
[x6(t)]])

In [6]:
robot.symbolic_input

Matrix([
[u1(t)],
[u2(t)]])

In [7]:
f = robot.getSymbolicF()
g = robot.getSymbolicG()
x = robot.symbolic_state
g1 = g[:,0]
g2 = g[:,1]


In [8]:
def lieDerivative(a, b, x):
    """ 
    returns the L_a(b)
    """
    return b.jacobian(x)*a

def lieBracket(a,b,x):
    return lieDerivative(a, b, x)-lieDerivative(b, a, x)

In [9]:
def isInDist(dist, vec):
    """
    Is vec in distribution dist?
    """
    d = dist[0].copy()
    for i in range(len(dist)-1):
        d = d.row_join(dist[i+1])
    rank1 = d.rank()
    d = d.row_join(vec)
    rank2 = d.rank()
    if rank2 > rank1:
        return False
    else:
        return True
    
def getDistRank(dist):
    """
    return the rank of distribution spanned by list of vecs in dist
    """
    d = dist[0].copy()
    for i in range(len(dist)-1):
        d = d.row_join(dist[i+1])
    return d.rank()

def getSmallestInvariantDistribution(dist, vec_fields, x):
    """
    Returns the smallest distribution invariant to vectors in the 
    vec_fields list and containing distribution spanned by the 
    vector fields in dist list. 
    """
    running = True
    result = []
    result +=dist
    while running:
        added_something = False
        for vec1 in result:
            for vec2 in vec_fields:
                vec3 = lieBracket(vec1, vec2, x)
                if not isInDist(result, vec3):
                    result +=[vec3]
                    added_something = True
        if not added_something:
            running = False
    
    return result
                    
        

In [10]:
result = getSmallestInvariantDistribution([g1,g2], [f,g1,g2],x)
len(result)

6

In [19]:
result

[Matrix([
 [         0],
 [sin(x_5)/m],
 [         0],
 [cos(x_5)/m],
 [         0],
 [         0]]),
 Matrix([
 [  0],
 [  0],
 [  0],
 [  0],
 [  0],
 [1/J]]),
 Matrix([
 [     sin(x_5)/m],
 [-x_6*cos(x_5)/m],
 [     cos(x_5)/m],
 [ x_6*sin(x_5)/m],
 [              0],
 [              0]]),
 Matrix([
 [  0],
 [  0],
 [  0],
 [  0],
 [1/J],
 [  0]]),
 Matrix([
 [ -2*x_6*cos(x_5)/m],
 [-x_6**2*sin(x_5)/m],
 [  2*x_6*sin(x_5)/m],
 [-x_6**2*cos(x_5)/m],
 [                 0],
 [                 0]]),
 Matrix([
 [              0],
 [ cos(x_5)/(J*m)],
 [              0],
 [-sin(x_5)/(J*m)],
 [              0],
 [              0]])]